In [1]:
#import numpy as np
#import re
import nltk
import pandas as pd

In [ ]:
#List of  proposed slang terms for entrainment project
terms = ['af', 'hmu', 'periodt', 'yeet','mood','lit' ,'bde', 'imo', 'goals','sus','thicc','woke','clutch',
         'weirdflexbutok','curve','tea','shook','dab','drip','tl;dr','salty','g.o.a.t','extra','turnt','yaas','thirsty','bruh','slay',
        'swol','finna','tfw']

In [ ]:
import yaml

with open('credentials.yaml', 'w') as output:
    data = {'app_key' : 'DafVCnQ6hVEPliMvCi6JcxugU', 'app_secret' : 'a5gVn9iMfI8ZIZka6prS72bedQqXaUjffDZNKuhmfdMPqnB3Zu',
           'oauth_token' : '165661752-mlqRJBECpDItueT00MMB3YqFkNoUpAnJAf0x6pak', 
            'oauth_token_secret' : 'JAHwk2K07BdCx0uqW3IWwSEj5ATnmF1GfjrjnNHce7MI3'
           }
    output.write(str(data))

In [ ]:
import json

# Enter your keys/secrets as strings in the following fields
credentials = {}  
credentials['CONSUMER_KEY'] = 'DafVCnQ6hVEPliMvCi6JcxugU'  
credentials['CONSUMER_SECRET'] = 'a5gVn9iMfI8ZIZka6prS72bedQqXaUjffDZNKuhmfdMPqnB3Zu'
credentials['ACCESS_TOKEN'] = '165661752-mlqRJBECpDItueT00MMB3YqFkNoUpAnJAf0x6pak'  
credentials['ACCESS_SECRET'] = 'JAHwk2K07BdCx0uqW3IWwSEj5ATnmF1GfjrjnNHce7MI3'

# Save the credentials object to file
with open("credentials.json", "w") as file:  
    json.dump(credentials, file)

In [2]:
from twython import Twython  

#data = {'app_key' : 'DafVCnQ6hVEPliMvCi6JcxugU', 'app_secret' : 'a5gVn9iMfI8ZIZka6prS72bedQqXaUjffDZNKuhmfdMPqnB3Zu',
           #'oauth_token' : '165661752-mlqRJBECpDItueT00MMB3YqFkNoUpAnJAf0x6pak', 
            #'oauth_token_secret' : 'JAHwk2K07BdCx0uqW3IWwSEj5ATnmF1GfjrjnNHce7MI3' }

APP_KEY = 'DafVCnQ6hVEPliMvCi6JcxugU'
APP_SECRET = 'a5gVn9iMfI8ZIZka6prS72bedQqXaUjffDZNKuhmfdMPqnB3Zu'

twitter = Twython(APP_KEY, APP_SECRET, oauth_version=2)
ACCESS_TOKEN = twitter.obtain_access_token()

In [ ]:
### Downloads tweets containing a user-specified query.
### The data is converted to an indexable of (JSON) tweets and written to disk.

import TwitterAPI
import yaml
import json

# User parameters.
TOTAL_COUNT= 1000                   # Total number of tweets to scrape (generally works up to ~15000)
COUNT=200                           # Twitter API allows 200 per scrape
CREDENTIALS = "credentials.yaml"    # YAML file containing developer credentials
SINK = "lit.json"                  # Output JSON file
QUERY = "lit -filter:retweet"      # Search term for scrape (note: further cleaning may be necessary
                                    # since the term may not be in the actual text of every tweet)

def main():
    # Creates API connection.
    with open(CREDENTIALS, "r") as source:
        credentials = yaml.load(source)
    api = TwitterAPI.TwitterAPI(
        consumer_key=credentials["app_key"],
        consumer_secret=credentials["app_secret"],
        access_token_key=credentials["oauth_token"],
        access_token_secret=credentials["oauth_token_secret"],
        )

    # Downloads data.
    # First download.
    kwargs = dict(
        q=QUERY, 
        lang="en", 
        count=COUNT,
        tweet_mode="extended",
    )
    data = []
    data.extend(api.request('search/tweets', kwargs))
    # This is the lowest ID for the batch, since they are given in reverse
    # chronological order. Since ID filtering is inclusive, we subtract one.
    max_id = data[-1]["id"] - 1
    while len(data) < TOTAL_COUNT:
        kwargs["max_id"]=max_id
        batch = []
        batch.extend(api.request('search/tweets',kwargs))
        if not batch: #no tweets returned
            break
        max_id = batch[-1]["id"] - 1
        data.extend(batch)
    
    #writes all tweets to disk
    print(f"{len(data)} tweets obtained")
    with open(SINK, "w") as sink:
        json.dump(data,sink)
    print(f"Data written to {SINK}")

if __name__ == "__main__":
    main()

In [3]:
#Creation of slang term dictionary from JSON file generated by API 
#If Tweet with slang term has a valid in reply to original tweet id, it is stored.

df = pd.read_json(path_or_buf = 'yeet.json', dtype = str)

reply_to_source = {}

for i,s in zip(list(df['full_text']), list(df['in_reply_to_status_id_str'])):
    if s !='None':
        reply_to_source[i] = s

In [ ]:
#Creating source json file of slang term that links back to a original tweet
twitter = Twython(APP_KEY, access_token=ACCESS_TOKEN) #list of dict

test_list = [] 
for ky in reply_to_source.values():
    test_list.extend(twitter.lookup_status(id=str(ky)))

with open('yeet_source.json', 'w') as file:
    json.dump(test_list, file)

In [19]:
#reading in the scrapped json, making a dict out of the reply text and original id string
df_1 = pd.read_json(path_or_buf ='yeet_source.json', dtype = str)

source_id = {}

for i,s in zip(list(df_1['text']), list(df_1['id_str'])):
    source_id[i] = s

In [20]:
#Reply tweets with link to original tweet via id
df_2 = pd.DataFrame(list(reply_to_source.items()), columns=['reply_tweet', 'original_tweet_id'])
#df_1

In [21]:
#original tweet with tweet id
df_3 = pd.DataFrame(list(source_id.items()), columns=['original_tweet', 'original_tweet_id'])
#df_3

In [22]:
#Combining Dataframes
final_df = pd.concat([df_3, df_2], axis = 1)
final_df = final_df.dropna()

final_df

,original_tweet,original_tweet_id,reply_tweet,original_tweet_id
0,I wonder if Della is gonna get added to the Du...,1128417439946027013,@saintwalker98 The way I see it either in seas...,1128417439946027013
1,When you log onto fortnite https://t.co/slBrRU...,1128409547968851971,Yeet,1128409547968851971
2,@kimsgcfs YEET,1128398385675816960,@PreciousJooniie YEET BACK,1128398385675816960
3,@SlanderOfficial @EDC_LasVegas @TrimmingRunePl8,1128417635148845056,@elchapBella @SlanderOfficial @EDC_LasVegas Ye...,1128417635148845056
4,@realelvenking |• jdhbkfdhnmkn yeah I do Y e e...,1128417276649254912,@MaiTheEnemy // I’m gonna YEET SOMEONE IN A SE...,1128417276649254912
5,@fiveeggshells @Zack_Real_MCDRP @redacted_yeet...,1128415147821809669,@SCP_682_TIT @Zack_Real_MCDRP @redacted_yeet @...,1128415147821809669
6,I really like the word “yeet” imma use it a li...,1128351486361124865,@amsuper8 Yeet,1128351486361124865
7,how similar is our music taste? (Stolen)\n\n10...,1128386242750631936,@ccfffee 40% yeet,1128386242750631936
8,@SCP_682_TIT @fiveeggshells @redacted_yeet @Sa...,1128415941128282112,@Zack_Real_MCDRP @fiveeggshells @redacted_yeet...,1128415941128282112
9,@SCP_682_TIT @Zack_Real_MCDRP @bobafettparody ...,1128415903727665153,@SautekhPhaeron @Zack_Real_MCDRP @bobafettparo...,1128415903727665153


In [9]:
#Tokenize elements in both lists, find the tag
#final_dict_af = {}
#final_dict_hmu = {}
#final_dict_periodt = {}
final_dict_yeet = {}
#final_dict_mood = {}
#final_dict_lit = {}

for o,r in zip(list(final_df['original_tweet']), list(final_df['reply_tweet'])):
    final_dict_yeet[o] = r

In [23]:
#print(final_dict_yeet)

#yeet_dict = pd.DataFrame(list(final_dict_yeet.items()))
#yeet_dict.columns = ['Original Tweet', 'Reply Tweet']
#yeet_dict

In [ ]:
#281
#len(final_dict_af)

#192
#len(final_dict_hmu)

#243
#len(final_dict_periodt)

#298
#len(final_dict_yeet)

#136
#len(final_dict_mood)

#185
#len(final_dict_lit)

In [26]:
#iterating over original tweets in slang dictionary for evidence of entrainment {original_tweet:reply_tweet}
#counter = 0

for i in final_dict_yeet.keys():
    #tokenization of tweet
    t = nltk.word_tokenize(i)
    print(t)
    for ch in t:
        s =  ch.lower()
        if 'yeet' in s:
            #counter += 1
            print(ch)
#print(counter)

['I', 'wonder', 'if', 'Della', 'is', 'gon', 'na', 'get', 'added', 'to', 'the', 'Duck', 'Tales', 'opening', 'credits', 'animation']
['When', 'you', 'log', 'onto', 'fortnite', 'https', ':', '//t.co/slBrRUcW9R']
['@', 'kimsgcfs', 'YEET']
YEET
['@', 'SlanderOfficial', '@', 'EDC_LasVegas', '@', 'TrimmingRunePl8']
['@', 'realelvenking', '|•', 'jdhbkfdhnmkn', 'yeah', 'I', 'do', 'Y', 'e', 'e', 't', '~', '&', 'lt', ';', '3']
['@', 'fiveeggshells', '@', 'Zack_Real_MCDRP', '@', 'redacted_yeet', '@', 'SautekhPhaeron', '@', 'Time_boi69', '@', 'GoldChannel9110', '@', 'OverseerZeta', 'I', '’', 'm', 'fine', 'with', 'that~']
redacted_yeet
['I', 'really', 'like', 'the', 'word', '“', 'yeet', '”', 'imma', 'use', 'it', 'a', 'little', 'more', '.']
yeet
['how', 'similar', 'is', 'our', 'music', 'taste', '?', '(', 'Stolen', ')', '10', '%', ':', 'Set', 'It', 'Off', '10', '%', ':', 'Neck', 'Deep', '10', '%', ':', 'State', 'Champs', '10', '%', ':', 'Blink', '182', '10', '%', ':', 'Wate…', 'https', ':', '//t.co/M1

In [ ]:
#The age of a certain term could affect the probabilty of it occuring in tweets

# == first round (600 tweets) ==

#af - 1/126 

#hmu - 1/72

#periodt - 2/51

#yeet - 10/75

#mood - 5/55

# ==second round (1000 tweets) == 

#af - 3/284

#hmu - 3/192

#periodt 5/243

#yeet 13/298

#mood 6/136 **1

#lit  9/185